In [176]:
import numpy as np
import pandas as pd
from src.utils import * 

In [177]:
gbsg = pd.read_csv("gbsg.csv")

# On a un jeu de données complet: 
np.where(gbsg.isna()==True)

(array([], dtype=int64), array([], dtype=int64))

On détermine $I$ et $J$: 

In [178]:
I, J = gbsg.shape

Trouver $K_1$, $K_2$

In [179]:
# Variables continues
idx_k1 = gbsg.columns[(gbsg.isin([0, 1])==False).any()]

Il faut effectuer l'encodage en variables dummies sur les données qui ont déjà des valeurs manquantes et non l'inverse ?

In [180]:
for i in gbsg.columns[(gbsg.isin([0, 1])).all()].to_numpy(): 
    gbsg[i+'_'] = 1-gbsg[i]

#Variables encodées en dummy variables 
idx_j = gbsg.columns[(gbsg.isin([0, 1])).all()]

In [181]:
data_missing = gbsg.mask(np.random.random(size=gbsg.shape)>0.5)

n_missing = data_missing.isna().sum().sum()
n_missing/(data_missing.shape[0]*data_missing.shape[1])

0.49834791059280853

In [182]:
data_missing.head()

,Unnamed: 0,pid,age,meno,size,grade,nodes,pgr,er,hormon,rfstime,status,meno_,hormon_,status_
0,1.0,NaN,NaN,NaN,18.0,2.0,NaN,0.0,0.0,NaN,1838.0,0.0,NaN,NaN,NaN
1,NaN,NaN,55.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,403.0,1.0,0.0,1.0,0.0
2,3.0,NaN,56.0,1.0,NaN,3.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.0
3,4.0,NaN,NaN,0.0,25.0,NaN,1.0,0.0,NaN,0.0,NaN,0.0,1.0,1.0,NaN
4,5.0,130.0,NaN,1.0,NaN,NaN,5.0,0.0,NaN,1.0,1855.0,0.0,0.0,NaN,1.0


In [183]:
data_missing.tail()

,Unnamed: 0,pid,age,meno,size,grade,nodes,pgr,er,hormon,rfstime,status,meno_,hormon_,status_
681,682.0,586.0,NaN,0.0,NaN,NaN,NaN,NaN,38.0,NaN,NaN,0.0,1.0,NaN,NaN
682,683.0,NaN,NaN,1.0,NaN,NaN,2.0,1356.0,NaN,NaN,1152.0,NaN,NaN,0.0,NaN
683,NaN,1525.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0.0,0.0,NaN,1.0
684,685.0,NaN,NaN,0.0,NaN,2.0,NaN,NaN,70.0,NaN,NaN,0.0,1.0,1.0,NaN
685,NaN,NaN,NaN,NaN,7.0,NaN,7.0,NaN,972.0,1.0,758.0,NaN,NaN,0.0,NaN


In [184]:
class IterativeFAMDImputer(FAMD):
    def __init__(self, data, k1, k2, n_components=2):
        super().__init__(data, k1, k2)
        """Initialisation
        Args:
            n_components (int, optional): _description_. Defaults to 2.
            data (_type_, optional): _description_. Defaults to None.
        """
        self.n_components = n_components
        
    def inital_impute(self, data):
        #Pour les variables continues
        self.sj = self.df_C0.std(axis=0).to_numpy()
        Ximp_C0 = self.df_C0.copy()
        for c in self.df_C0.columns.to_numpy(): 
            Ximp_C0[c] = Ximp_C0[c].fillna(self.df_C0[c].mean())  
        self.df_C0 = Ximp_C0

        # Pour les variables catégorielles 
        self.sqrt_pj = np.sqrt(self.df_categ.sum(axis=0)/self.df_categ.shape[0]).to_numpy()
        Ximp_categ = self.df_categ.copy() 
        Ximp_categ =Ximp_categ.fillna(np.sqrt(self.df_categ.sum(axis=0)/self.df_categ.shape[0]))  
        res = Ximp_categ.copy()
        for h in range(3):
            col = [Ximp_categ.columns[h],Ximp_categ.columns[h+3]]
            somme = Ximp_categ[col].sum(axis=1)
            for j in range(self.df.shape[0]):
                res.loc[j, col] = Ximp_categ[col].iloc[j]/somme[j]
        self.df_categ = res
        self.data_concat()
        pass 

    def ponderation_gsbs(self): 
        self.df_C0 = self.df[self.k1] # redefini df_C0 avec le df actuel
        self.sj = self.df_C0.std(axis=0).to_numpy()
        new_df_C0 = self.df_C0 / self.sj
        self.df_C0 = new_df_C0 # redefini df_categ avec le df actuel


        self.df_categ = self.df[self.k2]
        self.sqrt_pj = np.sqrt(self.df_categ.sum(axis=0)/self.df_categ.shape[0]).to_numpy()
        res = self.df_categ.copy()
        for h in range(3):
            col = [self.df_categ.columns[h],self.df_categ.columns[h+3]]
            somme = self.df_categ[col].sum(axis=1)
            for j in range(self.df.shape[0]):
                res.loc[j, col] = self.df_categ[col].iloc[j]/somme[j]
        self.df_categ = res
        self.data_concat() # mise a jour de df_categ

    
    def impute(self, n_it, tol=1e-4,verbose=False):
        # Initialisation 
        idx_NA = 1- self.df.isna().astype(int).to_numpy() # 1 si obs 0 sinon
        #Initial imputation 
        self.inital_impute(self.df)
        self.data_concat() # on construit les données avec df_C0 et df_categ remplis 

        diff = np.inf
        last_chap = np.inf*np.ones_like(idx_NA)
        i= 0
        while i < n_it and diff > tol: 
            #self.ponderation_gsbs()
            Z_p = self.step3() #Updating D, M already inside
            #print(Z_p)
            X_chap = (Z_p + self.M)@np.sqrt(self.D)
            df = pd.DataFrame(idx_NA*(self.df).to_numpy() + + (1- idx_NA)*X_chap.to_numpy(), columns=self.df.columns)
            self.df= df
            diff = ((X_chap - last_chap)**2).mean().mean()
            last_chap = X_chap
            i +=1 
            print(df)
        if i < n_it: 
            print('Converged in', i)
        else: 
              print('Maximum iterations reached')    
        pass 

In [185]:
famd_algo = FAMD(data=gbsg, k1=idx_k1, k2=idx_j)
famd_algo.run_famd()

,Unnamed: 0,pid,age,size,grade,nodes,pgr,er,rfstime,meno,hormon,status,meno_,hormon_,status_
0,0.008771,-0.002468,-0.009499,-0.082980,-0.257329,-0.277360,0.004535,0.003282,0.007345,-0.000033,-0.000016,-0.000353,0.000049,0.000008,0.000256
1,-0.049333,0.006886,-0.001839,0.232470,1.983031,0.744805,-0.024377,-0.018259,-0.025045,0.000023,-0.000122,0.001147,-0.000033,0.000059,-0.000831
2,-0.038476,-0.000366,-0.046770,-0.010843,1.986065,-0.087813,-0.018084,-0.014074,-0.006190,-0.000118,-0.000271,0.000204,0.000175,0.000131,-0.000148
3,-0.035535,-0.002396,-0.059458,-0.078940,1.991903,-0.320968,-0.016369,-0.012939,-0.000931,-0.000158,-0.000313,-0.000059,0.000233,0.000151,0.000042
4,0.005039,0.000135,0.006814,0.004339,-0.266804,0.021660,0.002354,0.001841,0.000608,0.000018,0.000038,-0.000016,-0.000026,-0.000018,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,-0.036566,-0.001547,-0.053924,-0.050462,1.979351,-0.223203,-0.016993,-0.013341,-0.003094,-0.000141,-0.000294,0.000050,0.000208,0.000142,-0.000036
682,0.008490,-0.001989,-0.006037,-0.066936,-0.279694,-0.221892,0.004325,0.003166,0.006180,-0.000023,-0.000004,-0.000294,0.000034,0.000002,0.000213
683,-0.035976,-0.001876,-0.055849,-0.061499,1.974498,-0.260846,-0.016661,-0.013116,-0.002221,-0.000147,-0.000300,0.000007,0.000217,0.000145,-0.000005
684,0.007932,-0.001826,-0.005382,-0.061442,-0.263826,-0.203500,0.004036,0.002957,0.005698,-0.000021,-0.000002,-0.000271,0.000030,0.000001,0.000196


In [186]:
ifamd = IterativeFAMDImputer(n_components=4, data=data_missing, k1=idx_k1, k2=idx_j)

In [187]:
ifamd.df

,Unnamed: 0,pid,age,meno,size,grade,nodes,pgr,er,hormon,rfstime,status,meno_,hormon_,status_
0,1.0,NaN,NaN,NaN,18.0,2.0,NaN,0.0,0.0,NaN,1838.0,0.0,NaN,NaN,NaN
1,NaN,NaN,55.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,403.0,1.0,0.0,1.0,0.0
2,3.0,NaN,56.0,1.0,NaN,3.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.0
3,4.0,NaN,NaN,0.0,25.0,NaN,1.0,0.0,NaN,0.0,NaN,0.0,1.0,1.0,NaN
4,5.0,130.0,NaN,1.0,NaN,NaN,5.0,0.0,NaN,1.0,1855.0,0.0,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,682.0,586.0,NaN,0.0,NaN,NaN,NaN,NaN,38.0,NaN,NaN,0.0,1.0,NaN,NaN
682,683.0,NaN,NaN,1.0,NaN,NaN,2.0,1356.0,NaN,NaN,1152.0,NaN,NaN,0.0,NaN
683,NaN,1525.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,0.0,0.0,NaN,1.0
684,685.0,NaN,NaN,0.0,NaN,2.0,NaN,NaN,70.0,NaN,NaN,0.0,1.0,1.0,NaN


$\textcolor{red}{\text{REGLER PROBLEME de ponderations! Il y t'il des ponderations entre étapes? On ponderee les categories? Qqchose marche pas car on a des valeurs négatives :(}}$

In [189]:
ifamd.impute(1000)

     Unnamed: 0          pid       age      size     grade     nodes  \
0      1.000000   967.368715  52.99697  18.00000  2.000000  4.795014   
1    324.681682   967.368715  55.00000  29.72619  2.117479  4.795014   
2      3.000000   967.368715  56.00000  29.72619  3.000000  4.795014   
3      4.000000   967.368715  52.99697  25.00000  2.117479  1.000000   
4      5.000000   130.000000  52.99697  29.72619  2.117479  5.000000   
..          ...          ...       ...       ...       ...       ...   
681  682.000000   586.000000  52.99697  29.72619  2.117479  4.795014   
682  683.000000   967.368715  52.99697  29.72619  2.117479  2.000000   
683  324.681682  1525.000000  52.99697  29.72619  2.117479  1.000000   
684  685.000000   967.368715  52.99697  29.72619  2.000000  4.795014   
685  324.681682   967.368715  52.99697   7.00000  2.117479  7.000000   

             pgr          er      rfstime      meno    hormon    status  \
0       0.000000    0.000000  1838.000000  0.544558  0.42055